In [1]:
import os
import sys
os.chdir('/Users/student/mass_spec/')

In [2]:
MzRt_files = [x for x in os.listdir('/Users/student/mass_spec/data/HirschhornLab_MetabolomicsData/') if '.MzRtInfo' in x]
[BioAge_file, MCDS_file, OE_file] = MzRt_files

In [3]:
import pandas as pd
os.chdir('/Users/student/mass_spec/data/HirschhornLab_MetabolomicsData/')
df_b = pd.DataFrame.from_csv(BioAge_file, sep = '\t')
df_m = pd.DataFrame.from_csv(MCDS_file, sep = '\t')
df_o = pd.DataFrame.from_csv(OE_file, sep = '\t')

In [6]:
df_trans = pd.read_excel('/Users/student/mass_spec/data/transformations.xlsx', sheetname = 'Common chemical relationships')
df_trans = df_trans.drop(0)
df_trans.set_index('Element', inplace = True)
df_trans.index.name = 'Reaction'
# df_trans = df_trans.reset_index(drop = True)
df_trans_mz = df_trans['Δm/z']
# df_trans_mz.rename(columns = {'Δm/z' : 'a'}, inplace = True)
df_trans_mz = pd.DataFrame(df_trans_mz)
df_trans_mz.rename(columns = {'Δm/z' : 'delta_m.z'}, inplace = True)

In [7]:
# Sequentially add columns to df_diff to get self-subtraction matrix
import numpy as np
# df_b['m.z']
df_diffs = pd.DataFrame()
df_diffs_temp = pd.DataFrame()
df_diffs = pd.DataFrame(df_b['m.z'].copy())
df_diffs_temp = df_diffs.copy()
for i in range(df_diffs.shape[0]):
# for i in range(0, 5):
    kwargs = {df_b['m.z'].index[i] : lambda df : df['m.z']}
    kwargs_temp = {df_b['m.z'].index[i] : lambda df : np.repeat(df['m.z'][i], df['m.z'].shape[0])}
    
    df_diffs = df_diffs.assign(**kwargs)
    df_diffs_temp = df_diffs_temp.assign(**kwargs_temp)
    
df_diffs = df_diffs.subtract(df_diffs_temp)
df_diffs = df_diffs.drop('m.z', axis = 1)

,C14:0_LPC_C8-pos,C16:1_LPC_C8-pos,C16:0_LPC,C18:2_LPC_C8-pos,C18:1_LPC_C8-pos
Metabolite,,,,,
C14:0_LPC_C8-pos,0.0000,-26.0156,-28.0311,-52.0313,-54.0469
C16:1_LPC_C8-pos,26.0156,0.0000,-2.0155,-26.0157,-28.0313
C16:0_LPC,28.0311,2.0155,0.0000,-24.0002,-26.0158
C18:2_LPC_C8-pos,52.0313,26.0157,24.0002,0.0000,-2.0156
C18:1_LPC_C8-pos,54.0469,28.0313,26.0158,2.0156,0.0000
C18:0_LPC_C8-pos,56.0626,30.0470,28.0315,4.0313,2.0157
C20:5_LPC,74.0133,47.9977,45.9822,21.9820,19.9664
C20:4_LPC_C8-pos,76.0317,50.0161,48.0006,24.0004,21.9848
C20:3_LPC,50.0130,23.9974,21.9819,-2.0183,-4.0339


In [19]:
TOLERANCE = 1.0
df_trans_test = df_trans_mz.copy()
df_trans_test = df_trans_test.set_value('Hydration', 'delta_m.z', -26.0156)
df_list = []
for i in range(len(df_diffs.columns)):
    diff_col = df_diffs[df_diffs.index[i]]
    index = pd.Index([x + '*****' + df_diffs.index[i] for x in diff_col.index], name = 'Transformation')
    columns = df_trans_mz.index
    df_mask = pd.DataFrame(np.isclose(diff_col.values[ : , None], df_trans_mz.values.transpose(), atol=TOLERANCE), 
                           index = index, columns = columns)
    df_list.append(df_mask)

df_mask = pd.concat(df_list)
    
    # https://stackoverflow.com/questions/39602004/can-i-use-pandas-dataframe-isin-with-a-numeric-tolerance-parameter
    # pd.DataFrame(np.isclose(df_diffs['1'].values[:, None], df_trans_test.values.transpose(), atol=1.0))
df_sum = df_mask.sum(axis = 1)
df_found = pd.DataFrame(df_sum[df_sum > 0], columns = ['Number Hits'])
df_found = df_found[df_found.index.map(lambda x : x.split('*****')[0] != x.split('*****')[1])]

In [7]:
# df_weights = pd.DataFrame(columns = ['Edge Weight'])
# for transformation in df_found.index:
#     reaction = df_mask.columns[df_mask.loc[transformation].values.flatten().tolist().index(True)]
#     df_trans_mz.loc[reaction]

#     # transformation_1 - transformation_2
#     transformation_1 = transformation.split(' - ')[0]
#     transformation_2 = transformation.split(' - ')[1]
#     diff = df_diffs[transformation_2][transformation_1]
#     reaction_diff = df_trans_mz.loc[reaction]
#     weight = abs(diff - reaction_diff).values[0]
#     df_weights = df_weights.append(pd.DataFrame([weight], index = [transformation], columns = ['Edge Weight']))

In [20]:
df_weights = pd.DataFrame(float('nan'), index = df_diffs.index, columns = df_diffs.columns)
for transformation in df_found.index:
    reaction = df_mask.columns[df_mask.loc[transformation].values.flatten().tolist().index(True)]
    df_trans_mz.loc[reaction]

    # transformation_1 - transformation_2
    transformation_1 = transformation.split('*****')[0]
    transformation_2 = transformation.split('*****')[1]
    diff = df_diffs[transformation_2][transformation_1]
    reaction_diff = df_trans_mz.loc[reaction]
    weight = abs(diff - reaction_diff).values[0]
    df_weights.set_value(transformation_1, transformation_2, 1.0 - weight)

df_weights = df_weights.fillna(value = 0.0)

In [21]:
# df_weights.shape
df_cyto = pd.melt(df_weights.reset_index(), id_vars = ['Metabolite'])
df_cyto = df_cyto.rename(columns = {'Metabolite' : 'Metabolite_from', 'variable' : 'Metabolite_to', 'value' : 'Weight'})
# pd.melt(df_weights)['variable']

In [22]:
df_cyto.to_csv('/Users/student/Google Drive/mass_spec/data/BioAge_cyto.csv', index = False)

In [23]:
df_cyto_pruned = df_cyto[df_cyto['Weight'] != 0.0]
df_cyto_pruned.to_csv('/Users/student/Google Drive/mass_spec/data/BioAge_cyto_pruned.csv', index = False)